3.2.1パーセプトロンのトレーニング

In [8]:
from sklearn import datasets
import numpy as np
# irisデータの取り出し。
iris = datasets.load_iris()
X = iris.data[:, [2,3]]
y = iris.target

# クラスラベルが文字列から整数値になっている。→計算高速化のため
print("Class labels:", np.unique(y))

Class labels: [0 1 2]


In [13]:
# from sklearn.cross_validation import train_test_split
# cross_validationを使ったらwarningが出たので、それにしたがい、model_selectionを使った。
from sklearn.model_selection import train_test_split
# 全体の30%をテストデータにする
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# 特徴量の標準化
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# トレーニングデータの平均と標準偏差を計算
sc.fit(X_train)
# 上記を使って標準化
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
